In [1]:
import pandas as pd
import numpy as np

In [2]:
df_r = pd.read_csv('record_corrected3.csv', low_memory=False).drop('Unnamed: 29', axis=1)
df_absent = pd.read_csv('absent_ships2.csv').rename(columns={'correct.ship_index/new ship_id': 'ship_index'}).drop(columns=['ship_id_old', 'wrong.ship_index', 'voyage_index', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])
df_r['ship_index'] = df_r['ship_index'].str.replace(" ", "", regex=False)
df_absent['ship_index'] = df_absent['ship_index'].str.replace(" ", "", regex=False)
df_s = pd.read_csv('ship.csv', low_memory=False).rename(columns={'id': 'ship_id'})
df_s_no_index = pd.read_csv('ship_by_id2.csv', low_memory=False).rename(columns={'id': 'ship_id'}).drop(columns=['Unnamed: 9', '328 ship names'])

In [3]:
df_s['ship_name'] = df_s['ship_name'].str.strip().str.title()
df_s_no_index['ship_name'] = df_s_no_index['ship_name'].str.strip().str.title()
df_absent['ship_name'] = df_absent['ship_name'].str.strip().str.title()
df_s['ship_code'] = df_s['ship_code'].str.strip().str.upper()
df_s_no_index['ship_code'] = df_s_no_index['ship_code'].str.strip().str.upper()
df_absent['ship_code'] = df_absent['ship_code'].str.strip().str.upper()
df_s_no_index['ship_index'] = np.nan
df_absent = df_absent.drop_duplicates(subset='ship_code')

In [143]:
# only_record = []
# for i in set(df[df['ship_index'].str.contains('/')]['ship_index']):
#     if len(df_absent[df_absent['ship_index']==i]) != 1:
#         only_record.append(i)

In [144]:
# only_ship = []
# for i in set(df_absent[df_absent['ship_index'].str.contains('/')]['ship_index']):
#     if len(df[df['ship_index']==i]) == 0:
#         only_ship.append(i)

In [145]:
for index, row in df_absent.iterrows():
    if '/' in row['ship_index'] and row['ship_index'].split('/')[0] == '':
        df_absent.at[index, 'ship_index'] = row['ship_index'].split('/')[1].strip()

In [146]:
for index, row in df_s_no_index.iterrows():
    match = df_absent[df_absent['ship_code']==row['ship_code']]
    if len(match) > 0:
        df_s_no_index.at[index, 'ship_index'] = match['ship_index'].iloc[0]

In [147]:
existing = list(set(df_s_no_index[df_s_no_index['ship_index'].notna()]['ship_index']))
df_s['ship_index'] = pd.to_numeric(df_s['ship_index'], errors='coerce').astype('Int64')
df_s['ship_index'] = df_s['ship_index'].astype(str)

In [148]:
df_test = df_s[~df_s['ship_index'].isin(existing)]
ignored = list(set(df_test[(df_test.duplicated(subset='ship_code', keep=False)) & (df_test['ship_code'] != '555')]['ship_code']))

In [149]:
for index, row in df_s_no_index.iterrows():
    if pd.isna(row['ship_index']) and row['ship_code'] not in ignored:
        match = df_test[df_test['ship_code'] == row['ship_code']]
        if not match.empty:
            df_s_no_index.at[index, 'ship_index'] = match['ship_index'].iloc[0]

In [161]:
res = []
a = list(df_s_no_index[df_s_no_index['ship_index'].isna()]['ship_code'])
for i in a:
    if i not in list(df_s['ship_code']):
        res.append(i)

In [165]:
df_s[df_s['ship_code'].isin(a)]

,ship_id,ship_name,ship_built_date,ship_built_place,ship_type,tonnage,guns,ship_index,ship_code
1,2,Aurore,NaN,NaN,NaN,NaN,NaN,1,AUR_B
50,51,Lys,NaN,NaN,NaN,NaN,NaN,50,LYS_A
60,61,Centaure,NaN,NaN,NaN,NaN,NaN,60,CEN_A
61,62,Lys,NaN,NaN,NaN,NaN,NaN,61,LYS_A
91,92,Massiac,NaN,NaN,NaN,NaN,NaN,91,MAS_A
112,113,Seine,NaN,NaN,NaN,NaN,NaN,112,SEI_A
132,133,Assemblée Nationale,NaN,NaN,NaN,NaN,NaN,132,ASN_A
151,152,Méduse,NaN,NaN,NaN,NaN,NaN,151,MED_A
154,155,Méduse,NaN,NaN,NaN,NaN,NaN,154,MED_A
161,162,Achille,NaN,NaN,NaN,NaN,NaN,161,ACH_B


In [164]:
a

['ACH_B',
 'ALO_A',
 'ASN_A',
 'AUR_B',
 'BAL_B',
 'BEL_A',
 'CEN_A',
 'CHM_A',
 'CLM_C',
 'CRL_A',
 'CVM_B',
 'DAU_G',
 'DDN_A',
 'DDO_C',
 'DDO_D',
 'DFR_A',
 'DIA_D',
 'GCR_B',
 'HIR_D',
 'LYS_A',
 'LYS_C',
 'LYS_D',
 'MAS_A',
 'MED_A',
 'MRS_C',
 'OIS_B',
 'SEI_A',
 'SER_A',
 'SYD_A',
 'UTL_D']